In [1]:
## To run as batch job


#imports:

# file imports

import numpy as np
import matplotlib.pyplot as plt
import gensim
import gensim.downloader
import os
from scipy.optimize import linprog
import nltk
from collections import defaultdict
from gensim.models import KeyedVectors
import sklearn
import scipy
import time
from multiprocessing import Pool
from nltk.corpus import stopwords
import json
from multiprocessing import Pool
import time

#nltk.download('stopwords')
#nltk.download('punkt')
# nltk.download('wordnet')


def sentence_preprocess(embed_dict, sentence,lowercase = 1, strip_punctuation = 1,  remove_stopwords = 1,removedigit = 1):
    ''' 1 : True, 0 : False : Lowercase, Strip puncutation, Remove Stopwords, removedigit'''

    stop_words = list(stopwords.words('english'))

    if lowercase == 1:
        sentence = sentence.lower()

    sentence_words = nltk.word_tokenize(sentence)

    if strip_punctuation == 1 and removedigit == 1:
        sentence_words = [word for word in sentence_words if word.isalpha()] 
        


    if remove_stopwords == 1:
        sentence_words = [word for word in sentence_words if not word in stop_words]
    
    ## to remove those words which are not in the embeddings that we have.
    
    sentence_words = [word for word in sentence_words if word in embed_dict.keys()]



    return sentence_words




embeddingtype = None
embd_model = None



## to load from embedding text files:
## have used this to load glove vectors and not word2vec

def load_glove(embeddingtype):
    
    if embeddingtype == 3:
        i = 300
    if embeddingtype == 4:
        i = 200
    if embeddingtype == 5:
        i = 100
    if embeddingtype == 6:
        i = 50
    
    
    embeddings_dict = defaultdict(lambda:np.zeros(i)) 
    # defaultdict to take care of OOV words.
    
    with open(f"../files/glove.6B.{i}d.txt",'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
        
    return embeddings_dict


def embeddings_setup(newembeddingtype):
    
    
    global embeddingtype
    global embd_model
    
    
    '''to avoid loading all the embeddings in the memory.'''
    
    ''''## Note : we are finding the embd matrix two times, ie once for each sentence in
        ## the pair of sentences.
        ## so this happens that embedding type is changed when find_embmatrix is called
        ## by the first sentence.
        The above line doesnt matter now as we not calling find_embmatrix , instead we setting up.
    '''
        
        
        
    if ( embeddingtype != newembeddingtype):
        #print("embdtype  entered :", embeddingtype != newembtype,"\n")
        #print("embd_model type changed to :", type(embd_model),"\n" )
        
        embeddingtype = newembeddingtype
        
        #embd_model = embeddings_setup(embeddingtype) #adictionary
        
        #print("embd_model type changed to :", type(embd_model),"\n" )
        #to make sure that we don't download the embeddings again and again,
        # we will check if the embedding type is same as the old one
        # and update global embd_model, vrna next time vo use hi nhi ho payega.
    
    
    
    
    
    if embeddingtype == 1:
        embedding = KeyedVectors.load('google300w2v.kv', mmap='r')
        ## This will be slower but will prevent kernel from crashing.
        
        ## comment the above line and uncomment this if you have sufficient RAM:
        
        #w2v_emb = gensim.downloader.load('word2vec-google-news-300')
        
    if embeddingtype == 2:
        print('Normalised word2vec not loaded, will get it soon')
        embedding = None
    
    if embeddingtype in (3,4,5,6):
        embedding = load_glove(embeddingtype)
        
    
    embd_model = embedding
    
    
        
def find_embdMatrix(sentence):
    global embeddingtype
    global embd_model
    #print(" global embedding type being passed is :", embeddingtype,"\n")
    #print("embedding type received by the find emb matrix is :", newembtype,"\n")
    #print("embd model type is :", type(embd_model),"\n")
    
    sent_mtx = []
    
    
    ##commented lines moved to embedding setup.
    
#     ''''## Note : we are finding the embd matrix two times, ie once for each sentence in
#     ## the pair of sentences.
#     ## so this happens that embedding type is changed when find_embmatrix is called
#     ## by the first sentence
#     '''
#     if ( embeddingtype != newembtype):
#         print("if embdtype part entered :", embeddingtype != newembtype,"\n")
        
#         embeddingtype = newembtype
#         embd_model = embeddings_setup(embeddingtype) #adictionary
        
#         print("embd_model type changed to :", type(embd_model),"\n" )
#     #to make sure that we don't download the embeddings again and again,
#     # we will check if the embedding type is same as the old one
#     # and update global embd_model, vrna next time vo use hi nhi ho payega.
    
    #print("embd_model type changed to :", type(embd_model),"\n" )
    for word in sentence:
        word_emb = embd_model[word]
        sent_mtx.append(word_emb)
    
    sent_mtx = np.array(sent_mtx).reshape(len(sentence),-1)

    return sent_mtx




def wasserstein_distance(pi, qj, D, cost = 'min'):
        """Find Wasserstein distance through linear programming
        p.shape=[m], q.shape=[n], D.shape=[m, n]
    
        suppose doc1 has m words and doc2 has n words, then an mxn array would be formed, 
        having distance of each word in doc1 to that of doc2.
    
    
    
        p.sum()=1, q.sum()=1, p∈[0,1], q∈[0,1]
        """
        A_eq = [] # a list which will later be converted to array after appending.
        for i in range(len(pi)): # len = number of words.
            A = np.zeros_like(D) # a 2d array made with the shape of D.  
            A[i, :] = 1 
            #print("Dshape, len pi till here :",D.shape,len(pi),"\n")
            
            # to make summation over "i" of Tij = pi, ie total / sum of outflow
            ## from one word is equal to its pi (normalized bag of word/ frequency/density)
            ## ex : if 2x3 D:
            ##T1,1 + T1,2 + T1,3 + 0 T2,1 + 0 T2,2 + 0 T2,3 = P1 and so on for every i,
            ## ie for each word in the doc1
            
            
            #print("A.shape", A.shape,"\n")
            A_eq.append(A.reshape(-1)) ## reshape(-1) flatens and then appending in A_eq.
            
            #print(A_eq,"Aeq\n")
            
            
            
            ## A_eq will be (m+n)x(m.n)
    
        for i in range(len(qj)):
            A = np.zeros_like(D)
            A[:, i] = 1 ## summation over "j" this time, so this time for different rows, 
            ## over a column "j" which refers to doc2, ie total incoming flow = qj density
            A_eq = list(A_eq)
            A_eq.append(A.reshape(-1))
            A_eq = np.array(A_eq)
        
        #print(A_eq.shape,A_eq)
       
        b_eq = np.concatenate([pi, qj])
        D = D.reshape(-1)
        #print("Dshape:",D.shape)
        if cost == 'max':
            D = D*(-1)
        
        result = linprog(D, A_eq=A_eq[:-1], b_eq=b_eq[:-1]) ## removing redundant to make 
        ## solution more robust.
        return np.absolute(result.fun), result.x , D.reshape((len(pi),len(qj)))  ## fun returns the final optimized value, x returns each value of xi,j that is the array

    
def relaxed_distance(pi,qj,D,cost='min'):
    
    # to find relaxed we just add the min/max cost directly using the least distance for pi to qj.
    
    # D is calculated from P to Q ie P in rows and Q in columns, To find Q to P we will transpose 
    if cost == 'min':
        p_to_q = np.dot(D.min(axis=1),pi)
        q_to_p = np.dot(D.T.min(axis=1),qj)
        
        return max(p_to_q,q_to_p)
    
    if cost == 'max':
        
        p_to_q = np.dot(D.max(axis=1),pi)
        q_to_p = np.dot(D.T.max(axis=1),qj)
        
        return min(p_to_q,q_to_p), None, D
        
        
    
    
class WMD:
    
    ''' wmd type = normal/relaxed, costtype = min/max.
    Enter Two sentence strings, cost = max if you want to try 
    max cost max flow version, embeddingtype = 1 for word2vec, 2 = normalized
    word2vec, 3 = glove300d, 4 = glove200d, 5 = glove100d 6 = glove50d'''
    
    def __init__(self,embeddingtype, wmd_type = 'normal', costtype='min'):
        
        
        self.cost = costtype
        
        self.embeddingtype = embeddingtype 
        self.wmd_type = wmd_type
        
        
        ## setting up the embeddings
        
        embeddings_setup(self.embeddingtype)
        
        
        
        
    #def word_count(self):
#         self.sent1_dic = defaultdict(int)
#         self.sent2_dic = defaultdict(int)
        
#         for word in sorted(sentence_preprocess(self.sent1)):
#             self.sent1_dic[word] += 1
            
#         for word in sorted(sentence_preprocess(self.sent2)):
#             self.sent2_dic[word] += 1
        
#         return dict(self.sent1_dic), dict(self.sent2_dic)



#     def wasserstein_distance(self, pi, qj, D):
#         """Find Wasserstein distance through linear programming
#         p.shape=[m], q.shape=[n], D.shape=[m, n]
    
#         suppose doc1 has m words and doc2 has n words, then an mxn array would be formed, 
#         having distance of each word in doc1 to that of doc2.
    
    
    
#         p.sum()=1, q.sum()=1, p∈[0,1], q∈[0,1]
#         """
#         A_eq = [] # a list which will later be converted to array after appending.
#         for i in range(len(pi)): # len = number of words.
#             A = np.zeros_like(D) # a 2d array made with the shape of D.  
#             A[i, :] = 1 
#             # to make summation over "i" of Tij = pi, ie total / sum of outflow
            ## from one word is equal to its pi (normalized bag of word/ frequency/density)
            ## ex : if 2x3 D:
            ##T1,1 + T1,2 + T1,3 + 0 T2,1 + 0 T2,2 + 0 T2,3 = P1 and so on for every i,
            ## ie for each word in the doc1
        
#             A_eq.append(A.reshape(-1)) ## reshape(-1) flatens and then appending in A_eq.
            ## A_eq will be (m+n)x(m.n)
    
#         for i in range(len(qj)):
#             A = np.zeros_like(D)
#             A[:, i] = 1 ## summation over "j" this time, so this time for different rows, 
#             ## over a column "j" which refers to doc2, ie total incoming flow = qj density
#             A_eq.append(A.reshape(-1))
#             A_eq = np.array(A_eq)
        
#         print(A_eq.shape,A_eq)
       
#         b_eq = np.concatenate([pi, qj])
#         D = D.reshape(-1)
#         if self.cost == 'max':
#             D = D*(-1)
        
#         result = linprog(D, A_eq=A_eq[:-1], b_eq=b_eq[:-1]) ## removing redundant to make 
#         ## solution more robust.
#         return result.fun, result.x  ## fun returns the final optimized value, x returns each value of xi,j that is the array

    
    def word_mover_distance(self,sentence1,sentence2):
        
        self.sent1 = sentence1
        #print(self.sent1 ,"\n")
        self.sent2 = sentence2
        #print(self.sent2 ,"\n")
        
        
        self.sent1_dic = defaultdict(int)
        self.sent2_dic = defaultdict(int)
        
        for word in sorted(sentence_preprocess(embd_model,self.sent1)): # sorted to have better
            self.sent1_dic[word] += 1 # idea of the sequence of the words. Creating BOW here
            
        for word in sorted(sentence_preprocess(embd_model,self.sent2)): #creating BOW from sorted sequence
            self.sent2_dic[word] += 1
        
        
        self.sent1_dic = dict(self.sent1_dic) # converted from default dict to dict.
        self.sent2_dic = dict(self.sent2_dic) # because following operations work on dict
        
        
        #print(self.sent1_dic ,"\n")
        #print(self.sent2_dic ,"\n")
        
        
        ## Now we will store a list/array of all the words in each sentence (in alphabetically sorted order)
        ## we will store corresponding count, and then corresponding Normalised count.
        self.sent1_words = np.array(list(self.sent1_dic.keys())) #dictionary keys converted to list than array
        self.sent1_counts = np.array(list(self.sent1_dic.values()))
        
        self.sent2_words = np.array(list(self.sent2_dic.keys()))
        self.sent2_counts = np.array(list(self.sent2_dic.values()))
        
        
        #print(self.sent1_words ,"\n")
        #print(self.sent1_counts ,"\n")
        
        #print(self.sent2_words ,"\n")
        #print(self.sent2_counts ,"\n")
        
        #dictionary values cant be converted into an array directly, hence the
        #list step.
        
        #print("embedding type being passed is :", self.embeddingtype,"\n")
        self.sent1_embmtx = find_embdMatrix(self.sent1_words)
        #print(self.sent1_embmtx.shape,"sent1emb\n")
        self.sent2_embmtx = find_embdMatrix(self.sent2_words)
        #print(self.sent2_embmtx.shape,"sent2emb\n")
        
        self.pi = self.sent1_counts/np.sum(self.sent1_counts) #NBOW step from BOW
        #print(self.pi,"self.pi\n")
        self.qj = self.sent2_counts/np.sum(self.sent2_counts)
        #print(self.qj,"self.qj\n")
        
        self.D = np.sqrt(np.square(self.sent1_embmtx[:, None] - self.sent2_embmtx[None, :]).sum(axis=2)) 
        #print(self.D.shape,"Dshape \n")
        ## programmers sought used mean instead of sum.
        ## scipy cdist can be used as well.
        
        if self.wmd_type == 'normal':
            return wasserstein_distance(self.pi, self.qj, self.D, self.cost)
        
        
        if self.wmd_type == 'relaxed':
            return relaxed_distance(self.pi,self.qj,self.D,self.cost)


## KNN

Train_BBCsport_sent = np.load("../files/Train_BBCsport_sent.npy")
Train_BBCsport_label = np.load("../files/Train_BBCsport_label.npy")
Test_BBCsport_sent = np.load("../files/Test_BBCsport_sent.npy")
Test_BBCsport_label = np.load("../files/Test_BBCsport_label.npy")


for i in range(10):
    print(Test_BBCsport_label[i],"\n",Test_BBCsport_sent[i])
    

print("##################Train details:\n")

for i in range(3):
    print(Train_BBCsport_label[i],"\n",Train_BBCsport_sent[i])


# embeddingtype = 3
# model = WMD(embeddingtype,wmd_type = 'relaxed', costtype='max')

            
    
no_testdocs = len(Test_BBCsport_sent)
no_testlabels = len(Test_BBCsport_label)
#no_testdocs,no_testlabels



actual_category = []
predicted_category = []

    



#import time
st = time.time()

embeddingtype = 3
model = WMD(embeddingtype,wmd_type = 'normal', costtype='min')


def predict_Category(sentence):
    
    distance_fromTrainset = []
    
    #for i in range (len(Train_BBCsport_sent)):
    for i in range (30):
        ## Find totalcost ie distance between sentence passed from test set to each sentence 
        ## in training set. and then append in the list.
        
        #print(sentence)
        #print(Train_BBCsport_sent[i])
        
        Totalcost, Tcoeff, Distancematx = model.word_mover_distance(sentence,Train_BBCsport_sent[i])
        print(Totalcost)
        distance_fromTrainset.append(Totalcost)
        
    distance_fromTrainset = np.array(distance_fromTrainset)
    #print('distance from train set array:',distance_fromTrainset)
    
    arr1indx = distance_fromTrainset.argsort()
    
    print("Distance and label sorted from test set",distance_fromTrainset[arr1indx[::1]], "\n",Train_BBCsport_label[arr1indx[::1]],"\n","Sentences: \n",Train_BBCsport_sent[arr1indx[::1]]) 
    
    
    ## Taking for different values of K
    
    #k = 5
    sorted_distance_fromTrainset_k5 = distance_fromTrainset[arr1indx[::1]][:5]
    sorted_labels_k5 = Train_BBCsport_label[arr1indx[::1]][:5]
    
    predicted_cat_k5 = scipy.stats.mode(sorted_labels_k5)[0]
    print("pred 5",predicted_cat_k5)
   

    #k = 7
    sorted_distance_fromTrainset_k7 = distance_fromTrainset[arr1indx[::1]][:7]
    sorted_labels_k7 = Train_BBCsport_label[arr1indx[::1]][:7]
    
    predicted_cat_k7 = scipy.stats.mode(sorted_labels_k7)[0]
    print("pred 7",predicted_cat_k7)

    #k = 11
    sorted_distance_fromTrainset_k11 = distance_fromTrainset[arr1indx[::1]][:11]
    sorted_labels_k11 = Train_BBCsport_label[arr1indx[::1]][:11]
    
    predicted_cat_k11 = scipy.stats.mode(sorted_labels_k11)[0]
    print("pred 11",predicted_cat_k11)

    #k = 15
    sorted_distance_fromTrainset_k15 = distance_fromTrainset[arr1indx[::1]][:15]
    sorted_labels_k15 = Train_BBCsport_label[arr1indx[::1]][:15]
    
    predicted_cat_k15 = scipy.stats.mode(sorted_labels_k15)[0]
    print("pred 15",predicted_cat_k15)

    #k = 21
    sorted_distance_fromTrainset_k21 = distance_fromTrainset[arr1indx[::1]][:21]
    sorted_labels_k21 = Train_BBCsport_label[arr1indx[::1]][:21]
    
    predicted_cat_k21 = scipy.stats.mode(sorted_labels_k21)[0]
    print("pred 21",predicted_cat_k21)


    #print(sorted_distance_fromTrainset,sorted_labels)

    
    return np.array([predicted_cat_k5,predicted_cat_k7,predicted_cat_k11,predicted_cat_k15,predicted_cat_k21])   
        
    
    
    
no_testdocs = len(Test_BBCsport_sent)
no_testlabels = len(Test_BBCsport_label)
#no_testdocs,no_testlabels



actual_categories = []
predicted_categories_list = []
for i in range (10):
    print(i,"\n", Test_BBCsport_label[i], "\n",Test_BBCsport_sent[i])
    
    
    actual_categories.append(Test_BBCsport_label[i]) 
    pred_category = predict_Category(Test_BBCsport_sent[i])
    print(pred_category)
    predicted_categories_list.append(pred_category)
    

et = time.time() 

print("time taken:",et-st)


    



   
 

 


cricket 
 ICC ponders one-day alterations

The rules of one-day cricket could be changed to allow the fielding side to take two wickets off a single ball.

A striking batsman could be dismissed lbw or caught and his partner run out. The International Cricket Council is also considering an idea to modify the 15-over period of fielding restrictions normally set for an innings start. Instead, the batting side would choose three different slots of five overs when only two fielders would be permitted to cover the boundaries. Sunil Gavaskar heads the ICC committee which has recommended the proposal in a bid to inject interest into the one-day game which the former India captain says has become "predictable". "We've looked at a double play situation," he explained in an interview with the Melbourne Age. "If a batsman's been given out lbw and the ball ricochets off to gully and the fielder picks it up and throws at the non-striker's end, so you have two dismissals off one ball. "Same if the ba

0 
 cricket 
 ICC ponders one-day alterations

The rules of one-day cricket could be changed to allow the fielding side to take two wickets off a single ball.

A striking batsman could be dismissed lbw or caught and his partner run out. The International Cricket Council is also considering an idea to modify the 15-over period of fielding restrictions normally set for an innings start. Instead, the batting side would choose three different slots of five overs when only two fielders would be permitted to cover the boundaries. Sunil Gavaskar heads the ICC committee which has recommended the proposal in a bid to inject interest into the one-day game which the former India captain says has become "predictable". "We've looked at a double play situation," he explained in an interview with the Melbourne Age. "If a batsman's been given out lbw and the ball ricochets off to gully and the fielder picks it up and throws at the non-striker's end, so you have two dismissals off one ball. "Same if th

<ipython-input-1-7f351a734d9e>:237: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
  result = linprog(D, A_eq=A_eq[:-1], b_eq=b_eq[:-1]) ## removing redundant to make
<ipython-input-1-7f351a734d9e>:237: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  result = linprog(D, A_eq=A_eq[:-1], b_eq=b_eq[:-1]) ## removing redundant to make
/home2/amit.pandey/miniconda3/lib/python3.9/site-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=1.09861e-18): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


6.582756105073719
6.007099567496299
6.594477251445178
7.04952180031447
6.395661586304511
6.647017873431175
6.251516460320107
Distance and label sorted from test set [6.00709957 6.15382289 6.25151646 6.39566159 6.41660937 6.43371348
 6.44837963 6.57843694 6.58275611 6.59447725 6.59906342 6.64701787
 6.66021761 6.66327039 6.68324833 6.70478023 6.71945809 6.72037207
 6.77130185 6.78123819 6.83206605 6.86087769 6.94785279 6.98167949
 6.99280224 7.01867551 7.0495218  7.06281029 7.13630406 7.29205937] 
 ['cricket' 'cricket' 'cricket' 'football' 'football' 'athletics' 'cricket'
 'rugby' 'football' 'rugby' 'football' 'cricket' 'football' 'athletics'
 'rugby' 'football' 'cricket' 'football' 'tennis' 'football' 'football'
 'football' 'football' 'athletics' 'football' 'football' 'rugby'
 'athletics' 'rugby' 'football'] 
 Sentences: 
 ['Batsman Law gains UK citizenship\n\nAustralian-born batsman Stuart Law has been granted British citizenship and will be able to play for Lancashire as a non-overse

6.9160044031134
6.397477020845595
6.607054366961081
6.356040006006824
6.4927639637118855
6.177583736722051
6.036607361817285
6.179901149756647
6.478305482059053
6.043740328359458
6.7432634150430655
5.942959160918388
6.866150043089193
6.029812461802569
6.799536744494357
6.436330863979971
6.404859123837603
6.2318439528805465
6.154455494689818
6.184353740285343
5.960927983099502
6.078249052615337
7.0478673119007595
6.063872703980834
6.432653896760947
6.089290270416562
6.72526345029639
5.908572654235307
6.283120319152759
6.186565780831333
Distance and label sorted from test set [5.90857265 5.94295916 5.96092798 6.02981246 6.03660736 6.04374033
 6.0638727  6.07824905 6.08929027 6.15445549 6.17758374 6.17990115
 6.18435374 6.18656578 6.23184395 6.28312032 6.35604001 6.39747702
 6.40485912 6.4326539  6.43633086 6.47830548 6.49276396 6.60705437
 6.72526345 6.74326342 6.79953674 6.86615004 6.9160044  7.04786731] 
 ['football' 'tennis' 'football' 'football' 'athletics' 'football'
 'football' 'ru

...

6.463785248912361
6.312775199619441
6.081091904004416
6.389353323047873
6.707952360945011
5.994561876641153
6.030937778011637
5.623535302513659
6.244571901793724
5.32751476558341
6.7255036225522815
5.739521406748969
6.755016419155216
5.589865299482186
6.403717928861366
5.748226755628474
5.837136604165841
5.724566931558715
5.826378751371658
5.793431639902591
5.853615216158929
5.510719064133912
6.75748212174541
5.99829001153526
6.296942158723022
5.567757263358492
6.45800232715257
5.252276262608089
5.9889031715328445
5.99930941907423
Distance and label sorted from test set [5.25227626 5.32751477 5.51071906 5.56775726 5.5898653  5.6235353
 5.72456693 5.73952141 5.74822676 5.79343164 5.82637875 5.8371366
 5.85361522 5.98890317 5.99456188 5.99829001 5.99930942 6.03093778
 6.0810919  6.2445719  6.29694216 6.3127752  6.38935332 6.40371793
 6.45800233 6.46378525 6.70795236 6.72550362 6.75501642 6.75748212] 
 ['football' 'football' 'rugby' 'rugby' 'football' 'football' 'football'
 'tennis' 'foot

7.486533261035108
7.515932676790417
7.465358111503203
7.449627837834138
7.469529850211566
6.8400591024017405
7.205658639472007
7.061536805870153
7.09298870449953
7.08597132645655
7.567961802415456
7.260243229059482
7.201618191600952
7.474633785897598
7.191873607008314
7.3852735735476305
7.428302687825124
7.073203580804463
7.161159054908574
6.900955182405969
6.983597207282621
6.997124078128956
6.869359238808266
7.389169931177245
7.3919930686617805
7.110170548248937
7.091413631720475
6.916414985387946
7.360600822088435
7.377481905976213
Distance and label sorted from test set [6.8400591  6.86935924 6.90095518 6.91641499 6.98359721 6.99712408
 7.06153681 7.07320358 7.08597133 7.09141363 7.0929887  7.11017055
 7.16115905 7.19187361 7.20161819 7.20565864 7.26024323 7.36060082
 7.37748191 7.38527357 7.38916993 7.39199307 7.42830269 7.44962784
 7.46535811 7.46952985 7.47463379 7.48653326 7.51593268 7.5679618 ] 
 ['football' 'football' 'football' 'football' 'football' 'rugby' 'football'
 'foot

6.151630398033309
6.94569548154201
6.310046900931122
6.294741618797154
6.879362753197338
5.9636631674117195
6.475708009399392
5.9702395956898044
6.777140153278081


/home2/amit.pandey/miniconda3/lib/python3.9/site-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=5.25729e-24): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


5.710082908202081
6.868299107674794
6.418770682753163
7.131883319823054
5.680305840278224
6.801994045973719
6.306092864913245
6.5558443572642915
5.7327809287848925
6.1892998305407065
6.053152395063587
5.825927023762156
6.200389748089457
6.934004160494224
5.1236902095869326
6.5671156993883715
6.091811649573561
7.080320146987963
5.622122854163543
5.999770497442627
6.5440122872406645
Distance and label sorted from test set [5.12369021 5.62212285 5.68030584 5.71008291 5.73278093 5.82592702
 5.96366317 5.9702396  5.9997705  6.0531524  6.09181165 6.1516304
 6.18929983 6.20038975 6.29474162 6.30609286 6.3100469  6.41877068
 6.47570801 6.54401229 6.55584436 6.5671157  6.77714015 6.80199405
 6.86829911 6.87936275 6.93400416 6.94569548 7.08032015 7.13188332] 
 ['football' 'football' 'football' 'football' 'football' 'football'
 'football' 'football' 'cricket' 'football' 'rugby' 'football' 'rugby'
 'rugby' 'athletics' 'football' 'football' 'tennis' 'athletics' 'cricket'
 'football' 'cricket' 'cric